# 15 单因子利率模型蒙卡模拟
* [15.1 简介](#15_1)
    * [15.1.1 单因子模型蒙卡抽样](#15_1_1)
    * [15.1.2 由抽样结果计算债券期权价格](#15_1_2)
* [15.2 蒙卡模拟步骤](#15_2)
    * [15.2.1 蒙卡抽样利率变化路径](#15_2_1)
    * [15.2.2 计算零息债券期权价格](#15_2_2)
* [15.3 步骤Python代码实现](#15_3)
* [15.4 计算示例](#15_4)
* [15.5 参考资料](#15_5)

## <a name="15_1"><a/> 15.1 简介
### <a name="15_1_1"><a/> 15.1.1 单因子模型蒙卡抽样
&nbsp; &nbsp; &nbsp; &nbsp; 在单因子模型下，瞬时无风险利率的变化过程为
    $$ dr(t) = (\theta(t)-ar(t))dt+\sigma dz \; .$$
其中$a$和$\sigma$为常数，$\theta(t)$由初始零息利率期限结构确定。
    
&nbsp; &nbsp; &nbsp; &nbsp; 如果我们考虑在$0$至$T$时间段内，利率$r(t)$的变化过程被离散化为$steps$次变化，在不同时间区间$r(t)$的取值为$r(t_i),\,t_i=i\times\Delta t,\, i=0,1,...,steps,\, \Delta t=T/(steps+1)$。$r(t_i)$由瞬时无风险利率变为$t_i$至$t_i=t_i+\Delta t$之间的短期无风险利率。离散化后的$r(t_i)$的变化过程可以表示为
    $$ r(t_i+\Delta t) = r(t_i)+(\theta_i-ar(t_i))\Delta t+\sigma\sqrt{\Delta t}\varepsilon\; .$$
$\varepsilon$为一标准正态分布随机数，$\theta_i$为由初始零息利率表示的变量，下面说明如何确定$\theta_i$。 

&nbsp; &nbsp; &nbsp; &nbsp; 由《期权、期货及其他衍生产品》书中第31章内容，我们知道在利率$r(t)$连续变化的情况下，
    $$\theta(t) = F_t(0, t)+aF(0,t)+\frac{\sigma^2}{2a}\left(1-e^{-2at}\;\;\right)\;.$$
其中$F(0,t)$和$F_t(0,t)$分别为$t$时刻的瞬时远期利率和$t$时刻瞬时远期利率的导数。  

&nbsp; &nbsp; &nbsp; &nbsp; 然后为了清晰方便起见，我们先统一将各种利率都用$R(t_0, t_1, t_2)$表示，其中$t_0$为观察时刻，$t_1$为利率作用开始时刻，$t_2$为结束时刻。此时初始零息利率$R(t)=R(0,0,t)$，离散化后的短期无风险利率$r(t_i)=R(t_i,t_i,t_i+\Delta t)$，短期远期利率$F(0,t_i)=R(0,t_i,t_i+\Delta t)$。考虑在连续复利时，
$$ \exp{[-R(0,0,t_i)\times t_i]}\cdot \exp{[-R(0,t_i,t_{i+1}\,)\times \Delta t]}=\exp{[-R(0,0,t_{i+1}\,)\times t_{i+1}\,]}\;,$$
得
$$ F(0, t_i) = R(0, t_i, t_i+\Delta t) = \frac{1}{\Delta t}(R(0, 0, t_i+\Delta t)\times(t_i+\Delta t)-R(0, 0, t_i)\times t_i) \;.$$ 
对于$F_t(0, t_i)$，可以向后取近似，
$$ F_t(0, t_i) = \frac{1}{\Delta t}(F(0, t_i)-F(0, t_{i-1}\;))\;.$$ 
由于在短期无风险利率$r(t_i)$的离散化变化过程中，$r(t_{i+1}\;)$为$t_{i+1}\,$至$t_{i+2}\,$之间的利率，$r(t_i\;)$为$t_i\,$至$t_{i+1}\,$之间的利率。发现建立两者变化关系时，将$\theta_i$表示为如下形式会比较合适，
$$ \theta_i =F_t(0, t_{i+1}\;)+aF(0,t_{i+1}\;)+\frac{\sigma^2}{2a}\left(1-e^{-2at_{i+1}}\quad\right)\;.$$
用初始零息利率可以表示为
$$\theta_i = \frac{1}{{\Delta t}^2}[R(t_{i}+2\Delta t)\times (t_i+2\Delta t)+R(t_i)\times t_i-2R(t_i-\Delta t)\times (t_i-\Delta t)]+\frac{a}{\Delta t}[R(t_i+2\Delta t)\times (t_i+2\Delta t)-R(t_i+\Delta t)\times(t_i+\Delta t)]+\frac{\sigma^2}{2a}
\left(1-e^{-2a(t_i+\Delta t)}\quad\right)\; .$$

&nbsp; &nbsp; &nbsp; &nbsp; 确定了$\theta_i$的表达式之后，我们就建立了$r(t_{i+1}\;)$和$r(t_i\,)$之间的递推关系。然后我们还需要确定初始利率$r(t_0)$，由$r(t_0)=R(0,t_0,t_0+\Delta t)=R(0,0,\Delta t)$,知$r(t_0)$即为初始零息利率曲线中$\Delta t$处的利率。这样我们就可以从$r(t_0)$开始，使用式$r(t_i+\Delta t)=r(t_i)+(\theta_i-ar(t_i))\Delta t+\sigma\sqrt{\Delta t}\varepsilon$和正态分布随机数$\varepsilon$逐步抽样出一条$r(t_i\,)$的变化路径。
    

### <a name="15_1_2"><a/> 15.1.2 由抽样结果计算债券期权价格
&nbsp; &nbsp; &nbsp; &nbsp; 考虑一在零息债券上的欧式看涨期权，债券的本金为$L$，期权的执行价格为$K$，期权的到期时间为$T_1$，债券的到期时间为$T_2$，$T_2>T_1$。该债券期权在当前的价格可以表示为
$$ C = \int_0^{r\_max}Q(r(T_1))\,\max{[L\cdot P(r(T_1))-K,\,0]}\,dr(T_1)\;.$$
其中$Q(r(T_1))$为由$t=0$至$t=T_1$所有可能的$r(t)$变化的路径下对应的贴现按该路径出现的概率进行加权平均所得值，$P(r(T_1))$为在$T_2$时刻到期的$1$美元零息债券在$T_1$时刻当瞬时无风险利率为$r(T_1)$时的价格。  

&nbsp; &nbsp; &nbsp; &nbsp; 如果我们想要使用蒙卡抽样出的一些$r(t)$由$t=0$至$t=T_2$的变化路径来计算该期权价格$C$。首先我们需要将$r(T_1)$的取值范围由$[0,r\_max]$划分为一些小的区间。然后找到$T_1$处于抽样路径中哪一个小的时间间隔，即找到$j$,使得$t_j\leq T_1 < t_{j+1}$。再将每条路径依其$r(t_j)$的大小处于哪一个$r(T_1)$取值区间进行分类，并近似认为同一类路径中的$r(t_j)$都是相同的。这时，
$$C\approx \sum\limits_{k=1}^{M}Q^\prime_k\, \max{(L\cdot P_k-K,\,0)}\; .$$
$M$为$r(T_1)$可能取值的区间的数量。$Q^\prime_k$为第$k$个区间对应的那些利率抽样路径中$t=0$至$t=T_1$部分的贴现的平均值乘以这些路径占总抽样数量的比例。$P_k$是第$k$个区间对应的那些路径中$t=T_1$至$t=T_2$部分的贴现的平均值。  
 
&nbsp; &nbsp; &nbsp; &nbsp; 如果我们将$r(T_1)$的取值区间划分得更加精细，然后增加利率路径的抽样数量，那么由上式所得的看涨期权价格将逐渐收敛于其解析结果。
    
## <a name="15_2"><a/> 15.2 蒙卡模拟步骤
### <a name="15_2_1"><a/> 15.2.1 蒙卡抽样利率变化路径
1. 假定$a$，$\sigma$，初始零息利率期限结构已知。我们将瞬时无风险利率$r(t)$由$t=0$至$t=T_2$之间的变化过程离散化为$steps$次变化，此时$\Delta t=T_2/(steps+1)$。$r(t_i)$为$t=i\times \Delta t$至$t=(i+1)\times \Delta t$之间的短期无风险利率。
1. 写一个函数R0t(rates, t)，输入初始零息利率期限结构rates和时间t，输出线性插值得到的期限为$t$的零息利率。在线性插值时，对于期限在已知的初始利率结构时间范围外的点，将其值设为最接近的利率。
1. 写一个函数Theta_ts(a, sigma, T, steps, rates)，输入相关参数，输出将在蒙卡抽样中被用到的所有的$\theta_i$的值。
$$\theta_i = \frac{1}{{\Delta t}^2}[R(t_{i}+2\Delta t)\times (t_i+2\Delta t)+R(t_i)\times t_i-2R(t_i-\Delta t)\times (t_i-\Delta t)]+\frac{a}{\Delta t}[R(t_i+2\Delta t)\times (t_i+2\Delta t)-R(t_i+\Delta t)\times(t_i+\Delta t)]+\frac{\sigma^2}{2a}
\left(1-e^{-2a(t_i+\Delta t)}\quad\right)\; .$$
1. 写一个蒙卡抽样函数，输入a, sigma, T, steps, theta_ts, R0, 输出一条蒙卡抽样利率变化路径。此处$R0$为$R(0,0,\Delta t)$，即利率初始取值。利率$r(t_i)$的递推关系为
    $$r(t_i+\Delta t)=r(t_i)+(\theta_i-ar(t_i))\Delta t+\sigma\sqrt{\Delta t}\varepsilon\;.$$ 使用该函数我们就可以进行蒙卡模拟抽样$r(t_i)$变化路径。
    
### <a name="15_2_2"><a/> 15.2.2 计算零息债券期权价格
1. 考虑期权期限为$T_1$，债券期限为$T_2$，期权执行价格为$K$，债券本金为$L$。设短期无风险利率$r(t)$在$t=0$至$t=T_2$之间变化$steps$次，将要抽取$num\_paths$条利率变化路径。此时$dt = T_2/(steps+1)$，并可以由$dt$确定一指标$tp$，使得$t_{tp}\leq T_1 < t_{tp+1}$。
1. 由上面所写的R0t()和Theta_ts()函数计算出$R0 = R0t(rates, \Delta t)$和theta_ts数组。
1. 将$r(T_1)$的取值区间由$[0,\,r\_max]$划分为等大小的$intervals$个区间，这里$r\_max$可以设为0.3，此时区间的大小为$dR=0.3/intervals$。
1. 初始化两个$intervals\,\times\, 2$大小的数组Qs和Ps。Qs留作存储每个$r(T_1)$取值区间内相应的路径数量和各路径上$t=0$至$t=T_1$之间的贴现值之和，Ps留作存储每个区间内路径数量和各路径上$t=T_1$至$t=T_2$之间贴现值之和。Ps和Qs内所有元素都初始化为0。
1. 由R0,theta_ts和相关参数，使用上面写的蒙卡抽样函数抽样出一条利率变化路径。根据$r(t_{tp})$的大小判断其位于$r(T_1)$的哪一个取值区间，并计算出该路径下$t=0$至$t=T_1$的贴现，$t=T_1$至$t=T_2$的贴现，更新Qs和Ps数组。重复这个过程$num\_paths$次。
1. 初始化看涨期权和看跌期权价格均为0。同时遍历Qs和Ps，如果区间$i$中路径数量$Ps[i][0]$不为零，则$Ps[i][1]/Ps[i][0]$为$T_1$时刻债券价格。如果该价格大于期权执行价格$K$，则将其减去$K$再乘以$Qs[i][1]/num\_paths$，并将结果加入看涨期权价格。如果债券价格小于$K$，类似计算并将结果加入看跌期权价格。最终得到的看涨期权价格和看跌期权价格即为蒙卡模拟计算出的零息债券期权价格。
    
## <a name="15_3"><a/> 15.3 步骤Python代码实现    

In [8]:
import numpy as np

INIT_RATES = [[3/365, 0.0501722], [31/365, 0.0498284], [62/365, 0.0497234], [94/365, 0.0496157],\
              [185/365, 0.0499058], [367/365, 0.0509389], [731/365, 0.0579733], [1096/365, 0.0630595], \
              [1461/365, 0.0673464], [1826/365, 0.0694816], [2194/365, 0.0708807], [2558/365, 0.0727527], \
              [2922/365, 0.0730852], [3287/365, 0.0739790], [3653/365, 0.0749015]]

def R0t(init_rates, t):
    # 由初始零息利率曲线插值得到t时刻零息利率。
    rates = list(init_rates)
    result = None
    if t <= rates[0][0]:
        result = rates[0][1]
    elif t >= rates[-1][0]:
        result = rates[-1][1]
    else:
        p = 0
        while rates[p][0] < t:
            p += 1
        result = rates[p-1][1]+(rates[p][1]-rates[p-1][1])/(rates[p][0]-rates[p-1][0])*(t-rates[p-1][0])
    return result

def Theta_ts(a, sigma, T, steps, init_rates):
    # 由相关参数计算出theta_ts数组。
    rates = list(init_rates)
    dt = T/(1.0+steps)
    result = []
    theta_t = None
    for i in range(1, steps+1, 1):
        ti = dt*i
        theta_t = sigma*sigma/2.0/a*(1.0-np.exp(-2.0*a*ti))
        theta_t += a*(R0t(rates, ti+dt)*(ti+dt)-R0t(rates, ti)*ti)/dt
        theta_t += ((ti+dt)*R0t(rates, ti+dt)+(ti-dt)*R0t(rates, ti-dt)-2.0*ti*R0t(rates, ti))/dt/dt
        result.append(theta_t)
    return result

def MC_one_factor_sample(a, sigma, T, steps, R0, theta_ts):
    # 抽样出一条利率变化路径。
    dt = T/(1.0+steps)
    result = [R0]
    Ri = R0
    for i in range(1, steps+1, 1):
        Ri = Ri+(theta_ts[i-1]-a*Ri)*dt+sigma*np.sqrt(dt)*np.random.normal()
        result.append(Ri)
    return result

def MC_bond_option_one_factor(K, L, T1, T2, a, sigma, steps, num_paths, intervals, init_rates):
    rates = list(init_rates)
    dt = T2/(1.0+steps)
    # tp为T1时刻所在时间区间的位置。
    tp = int(T1/dt)
    dt1 = T1-tp*dt
    dt2 = dt-dt1

    theta_ts = Theta_ts(a, sigma, T2, steps, rates)
    R0 = R0t(rates, dt)
    
    # r_max 设为0.3。
    dR = 0.3/intervals
    discounts = [[0.0, 0.0] for _ in range(intervals)]
    bond_prices = [[0.0, 0.0] for _ in range(intervals)]

    R_path = None
    discount_factor = None
    Rts_idx = None
    for i in range(num_paths):
        # 由于上面单独计算出了theta_ts，这里抽样路径时就不用每次都再计算。
        R_path = MC_one_factor_sample(a, sigma, T2, steps, R0, theta_ts)
        discount_factor = 0.0
        for j in range(steps, tp, -1):
            discount_factor += dt*R_path[j]
        discount_factor += dt2*R_path[tp]

        Rts_idx = int(R_path[tp]/dR)
        # 将可能超出范围的index调整回范围。
        Rts_idx = min(intervals-1, Rts_idx)
        Rts_idx = max(0, Rts_idx)
        bond_prices[Rts_idx][0] += 1.0
        bond_prices[Rts_idx][1] += np.exp(-discount_factor)
        
        discount_factor = dt1*R_path[tp]
        for j in range(tp-1, -1, -1):
            discount_factor += dt*R_path[j]
        discounts[Rts_idx][0] += 1.0
        discounts[Rts_idx][1] += np.exp(-discount_factor)

    bond_price = None
    call_price, put_price = 0.0, 0.0
    for i in range(intervals):
        if bond_prices[i][0] < 1.e-14:
            continue
        else:
            bond_price = L*bond_prices[i][1]/bond_prices[i][0]
            if bond_price > K:
                call_price += (bond_price-K)*discounts[i][1]/num_paths
            else:
                put_price += (K-bond_price)*discounts[i][1]/num_paths

    return call_price, put_price 

# <a name="15_4"><a/> 15.4 计算示例
&nbsp; &nbsp; &nbsp; &nbsp; 我们依旧参考《期权、期货及其他衍生产品》书中第31章例31-4，债券期权为零息债券上的看跌期权，期权执行时间为$T_1=3.0$，债券到期时间为$T_2=9.0$，期权执行价为$K=63.0$，债券本金为$L=100.0$。初始利率结构已知，Hull-White单因子模型中已知参数$a=0.1,\;\sigma=0.01$。

&nbsp; &nbsp; &nbsp; &nbsp; 然后利率变化次数$steps$设为200，抽样路径数量$num\_paths$设为20000，将$[0,\,0.3]$均匀划分为$intervals=200$个小区间作为$r(T_1)$的可能取值区间。此时我们可以进行一次模拟计算，并得到该欧式零息看涨期权和看跌期权的价格分别为1.01781和1.84377。

In [10]:
if __name__ == "__main__":
    a, sigma = 0.1, 0.01
    K, L = 63.0, 100.0
    T1, T2 = 3.0, 9.0
    steps = 200
    num_paths = 20000
    intervals = 200

    call_price, put_price = MC_bond_option_one_factor(K, L, T1, T2, a, sigma, steps, num_paths, intervals, INIT_RATES)
    print("\nCall, put prices: {0:.5f}  {1:.5f}".format(call_price, put_price))


Call, put prices: 1.01781  1.84377


## <a name="15_5"><a/> 15.5 参考资料
1. 《期权、期货及其他衍生产品》第31章，John C. Hull 著，王勇、索吾林译。